# Task 1: DATASET PREPARATION
---

# 1.0 Download and Import Essential Libraries

In [ ]:
# !pip install geemap

In [ ]:
import ee
import geemap
import pandas as pd
try:
    ee.Initialize(project='') # Make sure you have a Google Cloud Credential to Access Google Project
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project='') # Make sure you have a Google Cloud Credential to Access Google Project

# 2.0 Visualize Map with Image Collection Dataset

In [ ]:
# Initialize the map
Map = geemap.Map()
Map.add_basemap("HYBRID") # Add a basemap

# Add the ESA land cover layer
esa = ee.ImageCollection("ESA/WorldCover/v100").first()
esa_vis = {"bands": ["Map"]}
Map.addLayer(esa, esa_vis, "ESA Land Cover")
Map.add_legend(title="ESA Land Cover", builtin_legend="ESA_WorldCover")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [ ]:
# Define the feature collection for countries and filter for Africa
countries = ee.FeatureCollection(geemap.examples.get_ee_path("countries"))
africa = countries.filter(ee.Filter.eq("CONTINENT", "Africa"))
style = {"fillColor": "00000000"}
Map.addLayer(countries.style(**style), {}, "Countries", False)
Map.addLayer(africa.style(**style), {}, "Africa")
Map.centerObject(africa)
Map

Map(bottom=683.0, center=[41.244772343082076, 16.523437500000004], controls=(WidgetControl(options=['position'…

# 3.0 Dataset Preparation


## 3.1 Loading Respective Datasets and Cropland Extraction

In [ ]:
# Load the ESRI Global Land Cover ImageCollection
esri = ee.ImageCollection("projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m_TS") # Dataset from 2017 to 2023

# Define visualization parameters
esri_vis = {"min": 1, "max": 11, "palette": "esri_lulc"}

# Define the years of interest
years = [2017, 2018, 2019, 2020, 2021, 2022]

# Extract cropland and create a collection
def extract_cropland(img):
    return img.eq(5).clipToCollection(africa).selfMask()  # Assuming '5' is the cropland class

cropland_images = []
for year in years:
    esri_year = esri.filterDate(f"{year}-01-01", f"{year}-12-31").mosaic()
    # Map.addLayer(esri_year, esri_vis, f"ESRI LULC {year}")
    cropland_image = extract_cropland(esri_year)
    cropland_images.append(cropland_image)
    Map.addLayer(cropland_image, {"palette": ["#00A36C"]}, f"Cropland {year}")

Map

Map(bottom=1341.0, center=[-2.986927393334863, 18.984375000000004], controls=(WidgetControl(options=['position…

In [ ]:
# Combine cropland images into a single collection
collection = ee.ImageCollection.fromImages(cropland_images)
cropland_col = collection.map(lambda img: img.clipToCollection(africa).selfMask())
cropland_ts = cropland_col.toBands().rename([str(year) for year in years])

# Check the band names
print(cropland_ts.bandNames().getInfo())

['2017', '2018', '2019', '2020', '2021', '2022']


In [ ]:
# Retrieve and add climate data
era5 = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select(["temperature_2m", "total_precipitation"])
climate_images = []
for year in years:
    climate_year = era5.filterDate(f"{year}-01-01", f"{year}-12-31").mean().clipToCollection(africa)
    climate_images.append(climate_year)

In [ ]:
# Retrieve and add NDVI data
modis_ndvi = ee.ImageCollection("MODIS/006/MOD13A2").select("NDVI")
ndvi_images = []
for year in years:
    ndvi_year = modis_ndvi.filterDate(f"{year}-01-01", f"{year}-12-31").mean().clipToCollection(africa)
    ndvi_images.append(ndvi_year)

## 3.2 Respective CSV Datasets Preparation

<font color='orange'>1. Cropland Area (Sq km)

In [ ]:
# Compute cropland area by country for each year
cropland_area_values = []

for country in africa.toList(africa.size()).getInfo():
    country_name = country['properties']['NAME']
    country_geometry = ee.Feature(country).geometry()

    for year in years:
        cropland = cropland_images[years.index(year)]

        # Compute the cropland area
        cropland_area = cropland.multiply(ee.Image.pixelArea()).reduceRegion(
            reducer=ee.Reducer.sum(),
            geometry=country_geometry,
            scale=1000,
            maxPixels=1e13
        ).getInfo()

        if cropland_area:
            # Extract the correct key for cropland area
            cropland_area_key = list(cropland_area.keys())[0]
            cropland_area_value = cropland_area[cropland_area_key] / 1e6 # Sq km

            cropland_area_values.append({
                'country': country_name,
                'year': year,
                'cropland_area': cropland_area_value
            })

# Convert to a DataFrame and save as CSV
cropland_area_df = pd.DataFrame(cropland_area_values)
cropland_area_df.to_csv('cropland_area_by_country.csv', index=False)
cropland_area_df.head()

,country,year,cropland_area
0,Dem. Rep. Congo,2017,6802.801352
1,Dem. Rep. Congo,2018,8236.848528
2,Dem. Rep. Congo,2019,6202.692128
3,Dem. Rep. Congo,2020,5960.192934
4,Dem. Rep. Congo,2021,6982.951826


<font color='lightpink'>2. Temperature (K)</font>

In [ ]:
# Compute temperature mean by country for each year
temperature_mean_values = []

for country in africa.toList(africa.size()).getInfo():
    country_name = country['properties']['NAME']
    country_geometry = ee.Feature(country).geometry()

    for year in years:
        climate = climate_images[years.index(year)]

        # Compute mean temperature
        temp_mean = climate.select('temperature_2m').reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=country_geometry,
            scale=1000,
            maxPixels=1e13
        ).getInfo()

        if temp_mean:
            # Extract the correct key for temperature mean
            temp_mean_key = list(temp_mean.keys())[0]
            temp_mean_value = temp_mean[temp_mean_key]

            temperature_mean_values.append({
                'country': country_name,
                'year': year,
                'temperature_mean': temp_mean_value
            })

# Convert to a DataFrame and save as CSV
temperature_mean_df = pd.DataFrame(temperature_mean_values)
temperature_mean_df.to_csv('temperature_mean_by_country.csv', index=False)
temperature_mean_df.head()

,country,year,temperature_mean
0,Dem. Rep. Congo,2017,297.787228
1,Dem. Rep. Congo,2018,297.595389
2,Dem. Rep. Congo,2019,297.840254
3,Dem. Rep. Congo,2020,297.932522
4,Dem. Rep. Congo,2021,297.975652


<font color='green'>3. Normalized difference vegetation index (NDVI) [-1 to 1]</font>

In [ ]:
# Compute mean NDVI for cropland areas by country for each year
ndvi_mean_values = []

for country in africa.toList(africa.size()).getInfo():
    country_name = country['properties']['NAME']
    country_geometry = ee.Feature(country).geometry()

    for year in years:
        cropland = cropland_images[years.index(year)]
        ndvi = ndvi_images[years.index(year)]

        # Mask NDVI by cropland
        ndvi_cropland = ndvi.updateMask(cropland)

        # Compute mean NDVI for cropland areas
        ndvi_mean = ndvi_cropland.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=country_geometry,
            scale=1000,
            maxPixels=1e13
        ).getInfo()

        if ndvi_mean and ndvi_mean['NDVI'] is not None: # Check if ndvi_mean exists and has a valid 'NDVI' value
            # Extract the correct key for NDVI mean
            ndvi_mean_key = list(ndvi_mean.keys())[0]
            ndvi_mean_value = ndvi_mean[ndvi_mean_key] / 10000  # Scale the NDVI values to be between -1 and 1 (Based on documentation)

            ndvi_mean_values.append({
                'country': country_name,
                'year': year,
                'ndvi_mean': ndvi_mean_value
            })
        else:
            # Handle cases where no NDVI value is available
            ndvi_mean_values.append({
                'country': country_name,
                'year': year,
                'ndvi_mean': None # or a suitable default value
            })

# Convert to a DataFrame and save as CSV
ndvi_mean_df = pd.DataFrame(ndvi_mean_values)
ndvi_mean_df.to_csv('ndvi_mean_by_country_cropland.csv', index=False)
ndvi_mean_df.head()

,country,year,ndvi_mean
0,Dem. Rep. Congo,2017,0.585498
1,Dem. Rep. Congo,2018,0.588366
2,Dem. Rep. Congo,2019,0.589856
3,Dem. Rep. Congo,2020,0.603436
4,Dem. Rep. Congo,2021,0.594865


<font color='blue'>Average Annual Precipitation (mm)</font>

In [ ]:
# Retrieve and add annual precipitation data
era5_precip = ee.ImageCollection("ECMWF/ERA5_LAND/MONTHLY").select(["total_precipitation"])

# Compute average annual precipitation by country for each year
average_annual_precipitation_values = []

for country in africa.toList(africa.size()).getInfo():
    country_name = country['properties']['NAME']
    country_geometry = ee.Feature(country).geometry()

    for year in years:
        cropland = cropland_images[years.index(year)]

        # Filter the ERA5 collection for the specified year and sum the monthly precipitation
        annual_precipitation = era5_precip.filterDate(f"{year}-01-01", f"{year}-12-31").sum().clip(country_geometry)

        # Mask precipitation by cropland
        annual_precipitation_cropland = annual_precipitation.updateMask(cropland)

        # Compute the average precipitation for the cropland areas in the country
        precip_avg = annual_precipitation_cropland.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=country_geometry,
            scale=1000,
            maxPixels=1e13
        ).getInfo()

        if precip_avg:
            # Extract the correct key for precipitation average
            precip_avg_key = list(precip_avg.keys())[0]
            precip_avg_value = precip_avg[precip_avg_key] * 1000 * 25.4  # Convert from meters to millimeters

            average_annual_precipitation_values.append({
                'country': country_name,
                'year': year,
                'average_annual_precipitation_mm': precip_avg_value
            })

# Convert to a DataFrame and save as CSV
average_annual_precipitation_df = pd.DataFrame(average_annual_precipitation_values)
average_annual_precipitation_df.to_csv('average_annual_precipitation_by_country_cropland.csv', index=False)
average_annual_precipitation_df.head()

,country,year,average_annual_precipitation_mm
0,Dem. Rep. Congo,2017,1595.731927
1,Dem. Rep. Congo,2018,1622.158468
2,Dem. Rep. Congo,2019,1787.483950
3,Dem. Rep. Congo,2020,1825.378869
4,Dem. Rep. Congo,2021,1462.586482


<font color='brown'> 5. Soil Properties</font>
- `phh2o_mean`: Soil pH (No Units)
- `clay_mean`: Proportion of clay particles (< 0.002 mm) in the fine earth fraction (g/kg)
- `sand_mean`: Proportion of sand particles (> 0.05 mm) in the fine earth fraction (g/kg)
- `nitrogen_mean`: Total nitrogen (N) (cg/kg)
- `soc`: Soil organic carbon content in the fine earth fraction (dg/kg)

Soil Properties GEE Documentation: [Soil Grids 250m v2.0](https://gee-community-catalog.org/projects/isric/)

<font color='brown'>Notes: </font> For `nitrogen_mean` and `soc`, later decide whether need to change units to g/kg.

In [ ]:
# Define the SoilGrids pH image
isric_phh20 = ee.Image("projects/soilgrids-isric/phh2o_mean")

# Compute mean pH for cropland areas by country for each year
phh20_mean_values = []

for country in africa.toList(africa.size()).getInfo():
    country_name = country['properties']['NAME']
    country_geometry = ee.Feature(country).geometry()

    for year in years:
        cropland = cropland_images[years.index(year)]

        # Mask pH by cropland
        phh20_cropland = isric_phh20.updateMask(cropland)

        # Compute mean pH for cropland areas
        phh20_mean = phh20_cropland.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=country_geometry,
            scale=1000,
            maxPixels=1e13
        ).getInfo()

        if phh20_mean:
            # Extract the correct key for pH mean
            phh20_mean_key = list(phh20_mean.keys())[0]
            phh20_mean_value = phh20_mean[phh20_mean_key] / 10 # Conversion factor based on documentation

            phh20_mean_values.append({
                'country': country_name,
                'year': year,
                'phh2o_mean_value': phh20_mean_value
            })

# Convert to a DataFrame and save as CSV
phh20_mean_df = pd.DataFrame(phh20_mean_values)
phh20_mean_df.to_csv('phh2o_mean_by_country_cropland.csv', index=False)
phh20_mean_df.head()

,country,year,phh2o_mean_value
0,Dem. Rep. Congo,2017,5.408747
1,Dem. Rep. Congo,2018,5.401797
2,Dem. Rep. Congo,2019,5.416112
3,Dem. Rep. Congo,2020,5.438038
4,Dem. Rep. Congo,2021,5.435061


In [ ]:
# Define the SoilGrids clay image
isric_clay = ee.Image("projects/soilgrids-isric/clay_mean")

# Compute mean clay content for cropland areas by country for each year
clay_mean_values = []

for country in africa.toList(africa.size()).getInfo():
    country_name = country['properties']['NAME']
    country_geometry = ee.Feature(country).geometry()

    for year in years:
        cropland = cropland_images[years.index(year)]

        # Mask clay by cropland
        clay_cropland = isric_clay.updateMask(cropland)

        # Compute mean clay content for cropland areas
        clay_mean = clay_cropland.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=country_geometry,
            scale=1000,
            maxPixels=1e13
        ).getInfo()

        if clay_mean:
            # Extract the correct key for clay mean
            clay_mean_key = list(clay_mean.keys())[0]
            clay_mean_value = clay_mean[clay_mean_key] # Use default unit: g/kg

            clay_mean_values.append({
                'country': country_name,
                'year': year,
                'clay_mean_value': clay_mean_value
            })

# Convert to a DataFrame and save as CSV
clay_mean_df = pd.DataFrame(clay_mean_values)
clay_mean_df.to_csv('clay_mean_by_country_cropland.csv', index=False)
clay_mean_df.head()

,country,year,clay_mean_value
0,Dem. Rep. Congo,2017,335.819403
1,Dem. Rep. Congo,2018,317.736961
2,Dem. Rep. Congo,2019,334.971571
3,Dem. Rep. Congo,2020,332.487986
4,Dem. Rep. Congo,2021,320.117299


In [ ]:
# Define the SoilGrids sand image
isric_sand = ee.Image("projects/soilgrids-isric/sand_mean")

# Compute mean sand content for cropland areas by country for each year
sand_mean_values = []

for country in africa.toList(africa.size()).getInfo():
    country_name = country['properties']['NAME']
    country_geometry = ee.Feature(country).geometry()

    for year in years:
        cropland = cropland_images[years.index(year)]

        # Mask sand by cropland
        sand_cropland = isric_sand.updateMask(cropland)

        # Compute mean sand content for cropland areas
        sand_mean = sand_cropland.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=country_geometry,
            scale=1000,
            maxPixels=1e13
        ).getInfo()

        if sand_mean:
            # Extract the correct key for sand mean
            sand_mean_key = list(sand_mean.keys())[0]
            sand_mean_value = sand_mean[sand_mean_key] # Use default unit: g/kg

            sand_mean_values.append({
                'country': country_name,
                'year': year,
                'sand_mean_value': sand_mean_value
            })

# Convert to a DataFrame and save as CSV
sand_mean_df = pd.DataFrame(sand_mean_values)
sand_mean_df.to_csv('sand_mean_by_country_cropland.csv', index=False)
sand_mean_df.head()

,country,year,sand_mean_value
0,Dem. Rep. Congo,2017,430.235202
1,Dem. Rep. Congo,2018,462.290274
2,Dem. Rep. Congo,2019,431.951936
3,Dem. Rep. Congo,2020,435.046831
4,Dem. Rep. Congo,2021,456.527239


In [ ]:
# Define the SoilGrids nitrogen image
isric_nitrogen = ee.Image("projects/soilgrids-isric/nitrogen_mean")

# Compute mean nitrogen content for cropland areas by country for each year
nitrogen_mean_values = []

for country in africa.toList(africa.size()).getInfo():
    country_name = country['properties']['NAME']
    country_geometry = ee.Feature(country).geometry()

    for year in years:
        cropland = cropland_images[years.index(year)]

        # Mask nitrogen by cropland
        nitrogen_cropland = isric_nitrogen.updateMask(cropland)

        # Compute mean nitrogen content for cropland areas
        nitrogen_mean = nitrogen_cropland.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=country_geometry,
            scale=1000,
            maxPixels=1e13
        ).getInfo()

        if nitrogen_mean:
            # Extract the correct key for nitrogen mean
            nitrogen_mean_key = list(nitrogen_mean.keys())[0]
            nitrogen_mean_value = nitrogen_mean[nitrogen_mean_key] # Use default unit: cg/kg

            nitrogen_mean_values.append({
                'country': country_name,
                'year': year,
                'nitrogen_mean_value': nitrogen_mean_value
            })

# Convert to a DataFrame and save as CSV
nitrogen_mean_df = pd.DataFrame(nitrogen_mean_values)
nitrogen_mean_df.to_csv('nitrogen_mean_by_country_cropland.csv', index=False)
nitrogen_mean_df.head()

,country,year,nitrogen_mean_value
0,Dem. Rep. Congo,2017,2774.424658
1,Dem. Rep. Congo,2018,2668.407075
2,Dem. Rep. Congo,2019,2814.075162
3,Dem. Rep. Congo,2020,2825.262649
4,Dem. Rep. Congo,2021,2645.724268


In [ ]:
# Define the SoilGrids soc image
isric_soc = ee.Image("projects/soilgrids-isric/soc_mean")

# Compute mean soc content for cropland areas by country for each year
soc_mean_values = []

for country in africa.toList(africa.size()).getInfo():
    country_name = country['properties']['NAME']
    country_geometry = ee.Feature(country).geometry()

    for year in years:
        cropland = cropland_images[years.index(year)]

        # Mask soc by cropland
        soc_cropland = isric_soc.updateMask(cropland)

        # Compute mean soc content for cropland areas
        soc_mean = soc_cropland.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=country_geometry,
            scale=1000,
            maxPixels=1e13
        ).getInfo()

        if soc_mean:
            # Extract the correct key for soc mean
            soc_mean_key = list(soc_mean.keys())[0]
            soc_mean_value = soc_mean[soc_mean_key] # Use default unit: dg/kg

            soc_mean_values.append({
                'country': country_name,
                'year': year,
                'soc_mean_value': soc_mean_value
            })

# Convert to a DataFrame and save as CSV
soc_mean_df = pd.DataFrame(soc_mean_values)
soc_mean_df.to_csv('soc_mean_by_country_cropland.csv', index=False)
soc_mean_df.head()

,country,year,soc_mean_value
0,Dem. Rep. Congo,2017,408.538251
1,Dem. Rep. Congo,2018,395.489283
2,Dem. Rep. Congo,2019,415.028877
3,Dem. Rep. Congo,2020,417.660003
4,Dem. Rep. Congo,2021,388.627446


## 3.3 Merge All Geospatial Data into a Single Dataset

In [ ]:
# Load the individual CSV files
cropland_area_df = pd.read_csv('cropland_area_by_country.csv')
ndvi_mean_df = pd.read_csv('ndvi_mean_by_country_cropland.csv')
temperature_mean_df = pd.read_csv('temperature_mean_by_country.csv')
average_annual_precipitation_df = pd.read_csv('average_annual_precipitation_by_country_cropland.csv')
clay_mean_df = pd.read_csv('clay_mean_by_country_cropland.csv')
sand_mean_df = pd.read_csv('sand_mean_by_country_cropland.csv')
nitrogen_mean_df = pd.read_csv('nitrogen_mean_by_country_cropland.csv')
phh20_mean_df = pd.read_csv('phh2o_mean_by_country_cropland.csv')
soc_mean_df = pd.read_csv('soc_mean_by_country_cropland.csv')

In [ ]:
# Merge the dataframes on 'country' and 'year'
combined_df = pd.merge(cropland_area_df, ndvi_mean_df, on=['country', 'year'], how='outer')
combined_df = pd.merge(combined_df, temperature_mean_df, on=['country', 'year'], how='outer')
combined_df = pd.merge(combined_df, average_annual_precipitation_df, on=['country', 'year'], how='outer')
combined_df = pd.merge(combined_df, clay_mean_df, on=['country', 'year'], how='outer')
combined_df = pd.merge(combined_df, sand_mean_df, on=['country', 'year'], how='outer')
combined_df = pd.merge(combined_df, nitrogen_mean_df, on=['country', 'year'], how='outer')
combined_df = pd.merge(combined_df, phh20_mean_df, on=['country', 'year'], how='outer')
combined_df = pd.merge(combined_df, soc_mean_df, on=['country', 'year'], how='outer')

In [ ]:
# Check the dataframe
combined_df.head()

,country,year,cropland_area,ndvi_mean,temperature_mean,average_annual_precipitation_mm,clay_mean_value,sand_mean_value,nitrogen_mean_value,phh2o_mean_value,soc_mean_value
0,Dem. Rep. Congo,2017,6802.801352,0.585498,297.787228,1595.731927,335.819403,430.235202,2774.424658,5.408747,408.538251
1,Dem. Rep. Congo,2018,8236.848528,0.588366,297.595389,1622.158468,317.736961,462.290274,2668.407075,5.401797,395.489283
2,Dem. Rep. Congo,2019,6202.692128,0.589856,297.840254,1787.483950,334.971571,431.951936,2814.075162,5.416112,415.028877
3,Dem. Rep. Congo,2020,5960.192934,0.603436,297.932522,1825.378869,332.487986,435.046831,2825.262649,5.438038,417.660003
4,Dem. Rep. Congo,2021,6982.951826,0.594865,297.975652,1462.586482,320.117299,456.527239,2645.724268,5.435061,388.627446


In [ ]:
# Check the dimension of dataset
combined_df.shape

(306, 11)

In [ ]:
# Save the combined dataframes
combined_df.to_csv('combined_data.csv', index=False)

## 3.4 Fetch the Crop Yield Data

The crop yield data from Africa is sourced from [Our World in Data](https://ourworldindata.org/crop-yields#all-charts)

In [ ]:
# Load the crop yield data and check the dataframe
crop_yield_df = pd.read_csv('key-crop-yields.csv')
crop_yield_df.head()

,Entity,Code,Year,Wheat | 00000015 || Yield | 005419 || tonnes per hectare,Rice | 00000027 || Yield | 005419 || tonnes per hectare,Bananas | 00000486 || Yield | 005419 || tonnes per hectare,Maize | 00000056 || Yield | 005419 || tonnes per hectare,Soybeans | 00000236 || Yield | 005419 || tonnes per hectare,Potatoes | 00000116 || Yield | 005419 || tonnes per hectare,"Beans, dry | 00000176 || Yield | 005419 || tonnes per hectare","Peas, dry | 00000187 || Yield | 005419 || tonnes per hectare",Cassava | 00000125 || Yield | 005419 || tonnes per hectare,Cocoa beans | 00000661 || Yield | 005419 || tonnes per hectare,Barley | 00000044 || Yield | 005419 || tonnes per hectare
0,Afghanistan,AFG,1961,1.0220,1.5190,NaN,1.4000,NaN,8.666699,NaN,NaN,NaN,NaN,1.0800
1,Afghanistan,AFG,1962,0.9735,1.5190,NaN,1.4000,NaN,7.666700,NaN,NaN,NaN,NaN,1.0800
2,Afghanistan,AFG,1963,0.8317,1.5190,NaN,1.4260,NaN,8.133300,NaN,NaN,NaN,NaN,1.0800
3,Afghanistan,AFG,1964,0.9510,1.7273,NaN,1.4257,NaN,8.599999,NaN,NaN,NaN,NaN,1.0857
4,Afghanistan,AFG,1965,0.9723,1.7273,NaN,1.4400,NaN,8.800000,NaN,NaN,NaN,NaN,1.0857


In [ ]:
# Change the 'Entity' column into 'Country' and check all the unique values
crop_yield_df.rename(columns={'Entity': 'Country'}, inplace=True)
crop_yield_df['Country'].unique()

array(['Afghanistan', 'Africa', 'Africa (FAO)', 'Albania', 'Algeria',
       'Americas (FAO)', 'Angola', 'Antigua and Barbuda', 'Argentina',
       'Armenia', 'Asia', 'Asia (FAO)', 'Australia', 'Austria',
       'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh', 'Barbados',
       'Belarus', 'Belgium', 'Belgium-Luxembourg (FAO)', 'Belize',
       'Benin', 'Bhutan', 'Bolivia', 'Bosnia and Herzegovina', 'Botswana',
       'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi',
       'Cambodia', 'Cameroon', 'Canada', 'Cape Verde', 'Caribbean (FAO)',
       'Central African Republic', 'Central America (FAO)',
       'Central Asia (FAO)', 'Chad', 'Chile', 'China', 'China (FAO)',
       'Colombia', 'Comoros', 'Congo', 'Cook Islands', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Czechia',
       'Czechoslovakia', 'Democratic Republic of Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'East Timor',
       'Eastern Africa (FAO)', 'Eastern Asia (

In [ ]:
# Check the country name in combined data
combined_df['country'].unique()

array(['Dem. Rep. Congo', 'Chad', 'Central African Rep.', 'Tanzania',
       'Somalia', 'Kenya', 'Zimbabwe', 'Malawi', 'Mozambique', 'Burundi',
       'Madagascar', 'Eritrea', 'Ethiopia', 'Uganda', 'Rwanda',
       'S. Sudan', 'Mali', 'Mauritania', 'Benin', 'Niger', 'Togo',
       'Guinea', 'Guinea-Bissau', 'Liberia', 'Sierra Leone',
       'Burkina Faso', 'Gambia', 'W. Sahara', 'Gabon', 'Angola',
       'Tunisia', 'Algeria', 'Libya', 'South Africa', 'Botswana',
       'Namibia', 'Cameroon', 'Congo', 'Zambia', 'Djibouti', 'Somaliland',
       'Senegal', 'Nigeria', 'Ghana', "Côte d'Ivoire", 'Sudan', 'Morocco',
       'Egypt', 'Lesotho', 'eSwatini', 'Eq. Guinea'], dtype=object)

<font color='orange'>Important Observation: </font> Only `W. Sahara` and `Somalilan` do not have the crop yield data.

## 3.5 Data Engineering and Data Fusion

In [ ]:
# Rename the columns in the crop_yield_df
crop_yield_df.columns = [
    'country',
    'code',
    'year',
    'wheat_yield_tonnes_per_hectare',
    'rice_yield_tonnes_per_hectare',
    'bananas_yield_tonnes_per_hectare',
    'maize_yield_tonnes_per_hectare',
    'soybean_yield_tonnes_per_hectare',
    'potatoes_yield_tonnes_per_hectare',
    'beans_yield_tonnes_per_hectare',
    'peas_yield_tonnes_per_hectare',
    'cassava_yield_tonnes_per_hectare',
    'cocoa_beans_yield_tonnes_per_hectare',
    'barley_yield_tonnes_per_hectare',
]
crop_yield_df.head()

,country,code,year,wheat_yield_tonnes_per_hectare,rice_yield_tonnes_per_hectare,bananas_yield_tonnes_per_hectare,maize_yield_tonnes_per_hectare,soybean_yield_tonnes_per_hectare,potatoes_yield_tonnes_per_hectare,beans_yield_tonnes_per_hectare,peas_yield_tonnes_per_hectare,cassava_yield_tonnes_per_hectare,cocoa_beans_yield_tonnes_per_hectare,barley_yield_tonnes_per_hectare
0,Afghanistan,AFG,1961,1.0220,1.5190,NaN,1.4000,NaN,8.666699,NaN,NaN,NaN,NaN,1.0800
1,Afghanistan,AFG,1962,0.9735,1.5190,NaN,1.4000,NaN,7.666700,NaN,NaN,NaN,NaN,1.0800
2,Afghanistan,AFG,1963,0.8317,1.5190,NaN,1.4260,NaN,8.133300,NaN,NaN,NaN,NaN,1.0800
3,Afghanistan,AFG,1964,0.9510,1.7273,NaN,1.4257,NaN,8.599999,NaN,NaN,NaN,NaN,1.0857
4,Afghanistan,AFG,1965,0.9723,1.7273,NaN,1.4400,NaN,8.800000,NaN,NaN,NaN,NaN,1.0857


**Country Names that Required Changing Based on Combined Data**
- 'Democratic Republic of Congo' >>> 'Dem. Rep. Congo'
- 'Central African Republic' >>> 'Central African Rep.'
- 'Cote d'Ivoire' >>> 'Côte d'Ivoire'
- 'South Sudan' >>> 'S. Sudan'
- 'Equatorial Guinea' >>> 'Eq. Guinea'

In [ ]:
# Updating the entity names as requested
crop_yield_df['country'] = crop_yield_df['country'].replace({
    "Democratic Republic of Congo": "Dem. Rep. Congo",
    "Central African Republic": "Central African Rep.",
    "South Sudan": "S. Sudan",
    "Equatorial Guinea": "Eq. Guinea",
    "Cote d'Ivoire": "Côte d'Ivoire",
    "Eswatini": "eSwatini"
})

In [ ]:
# Merge the two dataframes based on 'Country' and 'Year'
merged_df_yield = pd.merge(combined_df, crop_yield_df, left_on=['country', 'year'], right_on=['country', 'year'])
merged_df_yield.head()

,country,year,cropland_area,ndvi_mean,temperature_mean,average_annual_precipitation_mm,clay_mean_value,sand_mean_value,nitrogen_mean_value,phh2o_mean_value,...,rice_yield_tonnes_per_hectare,bananas_yield_tonnes_per_hectare,maize_yield_tonnes_per_hectare,soybean_yield_tonnes_per_hectare,potatoes_yield_tonnes_per_hectare,beans_yield_tonnes_per_hectare,peas_yield_tonnes_per_hectare,cassava_yield_tonnes_per_hectare,cocoa_beans_yield_tonnes_per_hectare,barley_yield_tonnes_per_hectare
0,Dem. Rep. Congo,2017,6802.801352,0.585498,297.787228,1595.731927,335.819403,430.235202,2774.424658,5.408747,...,0.8416,3.7635,0.7719,0.4922,4.5702,0.5430,0.4046,8.1420,0.4861,0.6805
1,Dem. Rep. Congo,2018,8236.848528,0.588366,297.595389,1622.158468,317.736961,462.290274,2668.407075,5.401797,...,0.9009,3.7585,0.7702,0.5428,4.5620,0.5429,0.4048,8.1447,0.4516,0.6826
2,Dem. Rep. Congo,2019,6202.692128,0.589856,297.840254,1787.483950,334.971571,431.951936,2814.075162,5.416112,...,0.9692,3.7568,0.7704,0.5218,4.5737,0.5437,0.4058,8.1497,0.4797,0.6806
3,Dem. Rep. Congo,2020,5960.192934,0.603436,297.932522,1825.378869,332.487986,435.046831,2825.262649,5.438038,...,1.1184,3.7552,0.7706,0.5136,4.5909,0.5447,0.4047,8.1453,0.5077,0.6812
4,Dem. Rep. Congo,2021,6982.951826,0.594865,297.975652,1462.586482,320.117299,456.527239,2645.724268,5.435061,...,1.1130,3.7563,0.7697,0.5375,4.5889,0.5449,0.4050,8.1481,0.5207,0.6815


In [ ]:
# Checking the column names
merged_df_yield['country'].unique()

array(['Dem. Rep. Congo', 'Chad', 'Central African Rep.', 'Tanzania',
       'Somalia', 'Kenya', 'Zimbabwe', 'Malawi', 'Mozambique', 'Burundi',
       'Madagascar', 'Eritrea', 'Ethiopia', 'Uganda', 'Rwanda',
       'S. Sudan', 'Mali', 'Mauritania', 'Benin', 'Niger', 'Togo',
       'Guinea', 'Guinea-Bissau', 'Liberia', 'Sierra Leone',
       'Burkina Faso', 'Gambia', 'Gabon', 'Angola', 'Tunisia', 'Algeria',
       'Libya', 'South Africa', 'Botswana', 'Namibia', 'Cameroon',
       'Congo', 'Zambia', 'Djibouti', 'Senegal', 'Nigeria', 'Ghana',
       "Côte d'Ivoire", 'Sudan', 'Morocco', 'Egypt', 'Lesotho',
       'eSwatini', 'Eq. Guinea'], dtype=object)

In [ ]:
# Check the shape of the dataframe
merged_df_yield.shape

(294, 23)

In [ ]:
# Drop 'code' columns
merged_df_yield.drop(['code'], axis=1, inplace=True)

In [ ]:
# Check the data columns again
merged_df_yield.columns

Index(['country', 'year', 'cropland_area', 'ndvi_mean', 'temperature_mean',
       'average_annual_precipitation_mm', 'clay_mean_value', 'sand_mean_value',
       'nitrogen_mean_value', 'phh2o_mean_value', 'soc_mean_value',
       'wheat_yield_tonnes_per_hectare', 'rice_yield_tonnes_per_hectare',
       'bananas_yield_tonnes_per_hectare', 'maize_yield_tonnes_per_hectare',
       'soybean_yield_tonnes_per_hectare', 'potatoes_yield_tonnes_per_hectare',
       'beans_yield_tonnes_per_hectare', 'peas_yield_tonnes_per_hectare',
       'cassava_yield_tonnes_per_hectare',
       'cocoa_beans_yield_tonnes_per_hectare',
       'barley_yield_tonnes_per_hectare'],
      dtype='object')

In [ ]:
# Save the merged dataframe
merged_df_yield.to_csv('merged_data_yield.csv', index=False)